# Save a small version of the video with the trajectory on it

#### Written by Matan 20230306

In [5]:
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact
import time
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def l2n(x):
    return np.array(x)

def cropImage(img,x0,y0,width,height):
    return img[y0:y0+height,x0:x0+width,:]

def rebin(arr,new_shape):
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1],
             new_shape[2], arr.shape[2] // new_shape[2])
    
    return arr.reshape(shape).mean(-1).mean(1).mean(2)

def timeToSec(timeStr):#Transform m:s format into total number of seconds
    timeArr = timeStr.split(':')
    timeMin = int(timeArr[0])
    timeSec = int(timeArr[1])
    totSec = timeMin*60+timeSec
    return totSec

In [6]:
expSumFileName = 'C:\\Users\\User\\Documents\\Eden\\locate_git_code\\dress_exp\\dress_exp.csv'
expSum = pd.read_csv(expSumFileName,index_col=0)
#expSum=expSum[(expSum['date']==20231212)]
expSum=expSum.iloc[[145]]
#expSum=expSum.iloc[52]
print(expSum)

           operator  bottype  stars  bot_1_No  bot_2_No  bots  \
date                                                            
20240515.0     Eden  aligner    NaN       1.0       3.0   NaN   

            dress_diameter_bot_1  dress_diameter_bot_2  \
date                                                     
20240515.0                  23.0                  23.0   

                                        folder  \
date                                             
20240515.0  D:\Eden\dress_exp\16_5_24_aligners   

                                                    path  ... comments  \
date                                                      ...            
20240515.0  D:\Eden\dress_exp\16_5_24_aligners\C0810.MP4  ...      NaN   

            backedup  motorVal  production  linkingFileName  \
date                                                          
20240515.0       NaN       NaN         NaN              NaN   

            botLocatingFileName  botLinkingFileName miniVideoFile

In [7]:
smallVideoSuffix = '_small.mp4'

In [10]:
scaleDownFactor = 4
speedUpFactor = 3#4#5

width,height = 1450,1080
x0,y0 = 200,0
widthScaled = width//scaleDownFactor
heightScaled = height//scaleDownFactor

startTimeGlobal = time.time()


for index,row in expSum.iterrows():
    
    startTime = time.time()
   
    ############## Fetch experiments data #################
    fileNameVideo ='D:\\Eden\\dress_exp\\presentation_far_away_8_7_24\\C0894.MP4'#row.path
    fileNameLinked = 'D:\\Eden\\dress_exp\\7_5_24\\C0766.MP4_locatedFull20240508_bot_Eden_test.csv_linked.csv'#row.linkingFileName
    
    startMin ='0:00'#str(row.start_min)
    endMin ='0:25'#str(row.end_min)
    
    ############## load experiments files #################
    # Load video file
    video = cv2.VideoCapture(fileNameVideo)
    # Load locating file
    #df = pd.read_csv(fileNameLinked)
    #Load video Data
    fps = video.get(cv2.CAP_PROP_FPS) 
    
    ########### Filename to save small video #############
    outputFileName = fileNameVideo+smallVideoSuffix
    #Compute start and end frame
    startSec = timeToSec(startMin)
    endSec = timeToSec(endMin)
    startFrame = int(startSec*fps)
    endFrame = int(video.get(cv2.CAP_PROP_FRAME_COUNT))# int(endSec*fps)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(outputFileName,fourcc, fps, (widthScaled,heightScaled))
    video.set(cv2.CAP_PROP_POS_FRAMES, startFrame)
    frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
    ret = True
    print('Start scaling ' +str(endFrame-startFrame) +' frames down of video ' + fileNameVideo + '. Saved to '+ outputFileName)
    while ret and frameNum<endFrame:
        #video.grab()#captureFrame(video,frameNum)
        for i in range(speedUpFactor): #skip frames by speedUpFactor
            ret, img = video.read()

        if(ret): #if succesfffuly read an image
            frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
            #imgRed = img[:,:,0]

            #For large images, locate a cropped window
            imgCropped = cropImage(img,x0,y0,width,height)
            imgScaled = cv2.resize(imgCropped,(widthScaled,heightScaled),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            #Edge detection:
            out.write(imgScaled)
            #df = pd.concat([df,dft],ignore_index=True)

    video.release()
    out.release()

    endTime = time.time()
    duration = endTime-startTime
    print('Scaling and cropping '+ fileNameVideo + ' with '+ str(frameNum-startFrame) + ' frames took ' + str(duration) + ' seconds.')
endTimeGlobal = time.time()
durationGlobal = endTimeGlobal-startTimeGlobal

print('Scaling and crroping ' + str(len(expSum)) + ' videos took' + str(durationGlobal) + ' seconds')

Start scaling 876 frames down of video D:\Eden\dress_exp\presentation_far_away_8_7_24\C0894.MP4. Saved to D:\Eden\dress_exp\presentation_far_away_8_7_24\C0894.MP4_small.mp4
Scaling and cropping D:\Eden\dress_exp\presentation_far_away_8_7_24\C0894.MP4 with 876.0 frames took 4.521477460861206 seconds.
Scaling and crroping 1 videos took4.522480010986328 seconds


In [6]:
str(len(expSum))

'5'

In [7]:
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

widthScaled = width//scaleDownFactor
heightScaled = height//scaleDownFactor

ret, img = video.read()
imgCropped = cropImage(img,x0,y0,width,height)
startTime = time.time()
imgScaledI = cv2.resize(imgCropped,(widthScaled,heightScaled),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
endTime = time.time()
durationTime = endTime-startTime
print(durationTime*1E6)

startTime = time.time()
imgScaledN = rebin(imgCropped,(heightScaled,widthScaled,3)).astype(int)
endTime = time.time()
durationTime = endTime-startTime
print(durationTime*1E6)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(outputFileName)

In [ ]:
plt.imshow(imgScaled)

In [ ]:
10

In [ ]:
imgCropped.shape
imgScaled = rebin(imgCropped,(heightScaled,widthScaled,3)).astype(int)

In [ ]:
imgCropped.shape

In [ ]:
(widthScaled,heightScaled,3)

In [ ]:
3888000/300/3/270/4/3/1


In [ ]:
plt.imshow(imgScaled)


In [ ]:
imgC.shape[0]

In [ ]:
img  = captureFrame(video,frameNum)
imgCropped =cropImage(img,300,0,1200,int(height))

In [ ]:
ap = cv2.VideoCapture('C:/New folder/video.avi')
 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 5, (1280,720))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()